## 8.1階層式索引

In [1]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.Series(np.random.randn(9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                      [1,2,3,1,3,1,2,2,3]])
data

In [3]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           labels=[[0, 0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 2, 0, 1, 1, 2]])

In [4]:
# 選擇子集
data[['b','c']]  # 同 data['b':'c']

b  1    1.438287
   3    1.377960
c  1   -0.907391
   2   -0.993755
dtype: float64

In [6]:
# 選擇內層
data.loc[:,2]

a    0.380326
c   -0.993755
d    0.359150
dtype: float64

In [7]:
'''
重排DataFrame資料
'''
# 將資料的行"旋轉"為列
data.unstack()  # 設操作為最內層

,1,2,3
a,0.890653,0.380326,-0.899443
b,1.438287,NaN,1.377960
c,-0.907391,-0.993755,NaN
d,NaN,0.359150,0.997621


In [10]:
# 將資料的列"旋轉"為行
data.unstack().stack()   # 設操作為最內層

a  1    0.890653
   2    0.380326
   3   -0.899443
b  1    1.438287
   3    1.377960
c  1   -0.907391
   2   -0.993755
d  2    0.359150
   3    0.997621
dtype: float64

In [11]:
'''
每個軸都有自己的階層式索引
'''
frame = pd.DataFrame(np.arange(12).reshape((4,3)),
                    index=[['a', 'a', 'b', 'b'], [1,2,1,2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [12]:
# 給階層式索引的每一層名字
frame.index.names =['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [31]:
'''
利用 Pandas MultiIndex，可以新建立多維資料，效果同上
'''
columns2 = pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                                   names=['state', 'color'])
columns2

MultiIndex(levels=[['Colorado', 'Ohio'], ['Green', 'Red']],
           labels=[[1, 1, 0], [0, 1, 0]],
           names=['state', 'color'])

In [32]:
index2 = pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1,2,1,2]],
                                   names=['key1', 'key2'])
index2

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['key1', 'key2'])

In [33]:
frame2 = pd.DataFrame(np.arange(12).reshape((4,3)), index=index2, columns=columns2)
frame2

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### 重排階層及依階層排序值

In [34]:
# 針對兩個階層(軸)
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [39]:
# 用於指定層index
frame.sort_index(level=1)   # level=1: 第二層

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [41]:
frame.swaplevel(0,1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [40]:
frame.swaplevel(0,1).sort_index(level=0)   # level=0: 第一層

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 指定階層統計資訊

In [42]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [43]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 用DataFrame 的欄當 index

In [45]:
frame = pd.DataFrame({'a': range(7),
                      'b': range(7,0,-1),
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd': [0,1,2,0,1,2,3]
                     })
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [46]:
# 新物件拿原物件的一或多個欄位來當index
# columns --> index
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [51]:
# 選擇留下欄位資料(預設會被刪掉)
frame3 = frame.set_index(['c', 'd'], drop=False)
frame3

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [52]:
# index --> columns
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


---

## 8.2 合併資料集合

In [6]:
import pandas as pd
import numpy as np
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                   'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                   'data2': range(3)})
print(df1, '\n---\n', df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6 
---
   key  data2
0   a      0
1   b      1
2   d      2


In [7]:
'''
多對一 join
'''
# 未指定哪一欄做join
pd.merge(df1, df2)

# merge後，c、d欄位不見，因為預設是 inner join，可選擇 how='outer' 做聯集

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [8]:
# 指定欄位做join
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [9]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                   'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                   'data2': range(3)})

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [10]:
# 聯集
pd.merge(df1, df2, how = 'outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [11]:
'''
多對多 join
'''
df5 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
df6 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                   'data2': range(5)})
print(df5, '\n---\n', df6)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5 
---
   key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4


In [12]:
pd.merge(df5, df6, on='key', how='left')

# 產生列的笛卡兒積

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [13]:
pd.merge(df5, df6, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [14]:
# 合併多個key
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})

In [15]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [16]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [17]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [18]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [19]:
# 處理欄名重複:suffixes
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [24]:
# 用 DataFrame中的 index 用來作合併的 key
pd.merge(df3, df4, left_index=True, right_index=True)

,lkey,data1,rkey,data2
0,b,0,a,0
1,b,1,b,1
2,a,2,d,2


### 依index作合併

In [20]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                     'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [21]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [5]:
right1

,group_val
a,3.5
b,7.0


In [6]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [7]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [4]:
# 階層式index合併
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Neveda'],
                     'key2': [2000, 2001, 2002, 2001, 2002],
                     'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6,2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                            [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [6]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Neveda,2002,4.0


In [5]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [8]:
# 用list寫明要用哪些欄位為key進行合併
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [9]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Neveda,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [10]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                    index=['a', 'c','e'],
                    columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                     index=['b', 'c', 'd', 'e'],
                     columns=['Missouri', 'Albama'])

In [11]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [12]:
right2

,Missouri,Albama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [15]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Albama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [16]:
#用 DataFrame 的 join
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Albama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [22]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [23]:
# 以index對index進行合併
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                      index=['a', 'c', 'e', 'f'],
                      columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [25]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Albama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [27]:
left2.join([right2, another], how='outer')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6369: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,Ohio,Nevada,Missouri,Albama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


### 延軸做連接

In [6]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [7]:
np.concatenate((arr, arr), axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [10]:
# 軸標籤雖可以多樣的連接，但也有許多要考慮的問題
# ex1. index均不同的 Series
s1 = pd.Series([0,1], index=['a', 'b'])
s2 = pd.Series([2,3,4], index=['c', 'd', 'e'])
s3 = pd.Series([5,6], index=['f', 'g'])
s1, s2, s3

(a    0
 b    1
 dtype: int64,
 c    2
 d    3
 e    4
 dtype: int64,
 f    5
 g    6
 dtype: int64)

In [11]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [12]:
# 指定axis=1 --> 變成 DataFrame
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [14]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [15]:
pd.concat([s1,s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [16]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [17]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

TypeError: concat() got an unexpected keyword argument 'join_axes'

In [19]:
pd.__version__
# join_axes 自 pandas 0.25.0 版已 deprecated

'1.3.3'

In [20]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [21]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [22]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])  # axis=1 --> keys會變成欄標題

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [2]:
df1 = pd.DataFrame(np.arange(6).reshape((3,2)), index=['a', 'b', 'c'], columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape((2,2)), index=['a', 'c'], columns=['three', 'four'])

In [3]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [4]:
df2

,three,four
a,5,6
c,7,8


In [5]:
# 傳入list
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [6]:
# 改傳入dict物件
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [7]:
# 階層索引的建立
# concat函式參數：names---軸階層名稱
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [8]:
'''列index與資料完全不相關'''
df3 = pd.DataFrame(np.random.randn(3,4), columns=['a', 'b', 'c', 'd'])
df4 = pd.DataFrame(np.random.randn(2,3), columns=['b', 'd', 'a'])

In [9]:
df3

,a,b,c,d
0,-1.234658,1.319520,-0.157682,1.504395
1,1.353223,-1.673568,0.128359,-3.028662
2,-1.035482,1.511645,-0.734699,-0.059893


In [10]:
df4

,b,d,a
0,-1.533143,1.432892,-2.237628
1,0.890405,-2.303957,-0.046762


In [12]:
pd.concat([df3, df4], ignore_index=True)

,a,b,c,d
0,-1.234658,1.319520,-0.157682,1.504395
1,1.353223,-1.673568,0.128359,-3.028662
2,-1.035482,1.511645,-0.734699,-0.059893
3,-2.237628,-1.533143,NaN,1.432892
4,-0.046762,0.890405,NaN,-2.303957


### 合併有重複的資料

In [16]:
'''合併資料時，無法用merge或concat解決
'''
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.], index=['a', 'b', 'c', 'd', 'e', 'f'])

In [17]:
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [18]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [21]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [25]:
np.where(pd.isnull(b), a, b)

array([0. , 2.5, 2. , 3.5, 4.5, 5. ])

In [26]:
b.combine_first(a)  # 對齊

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

In [27]:
A = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                  'b': [np.nan, 2., np.nan, 6.],
                  'c': range(2, 18, 4)})
B = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                  'b': [np.nan, 3., 4., 6., 8.]})

In [28]:
A

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [29]:
B

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [30]:
A.combine_first(B)  # 從 B 中取得值，去修補 A 自身的遺失值

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


---
## 8.3 重塑(reshape)和旋轉(pivot)

###  用階層式index進行重塑

In [32]:
'''
  stack: 將欄"旋轉"成列
unstack: 將列"旋轉"成爛
'''
data = pd.DataFrame(np.arange(6).reshape((2,3)),
                   index=pd.Index(['Ohio', 'Colorado'], name='state'),
                   columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [35]:
result = data.stack()
result   # 維度變成一維Series
# number旋轉為行索引，並且置於行索引的最內層（外層為索引state）

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [36]:
result.unstack()   # unstack操作預設將內層索引 number(最低級別) 旋轉為列索引

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [39]:
# 指定層編號或名稱以指定變動哪一層(一般:最裡面的index不會被旋轉)
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [38]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [41]:
# 當某層缺少值
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [42]:
data2.unstack()   # 有遺失值

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [43]:
data2.unstack().stack()  # 自動過濾遺失值

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [44]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [45]:
df = pd.DataFrame({'left': result, 'right': result+5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [46]:
df.unstack('state')   # unstack的那層，會變成最下層

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [47]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### "長格式"旋轉成"寬格式"

In [3]:
import requests   # 使用 requests 發送 GET 請求

request_url = "https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/examples/macrodata.csv"
resp = requests.get(request_url)
# print(resp.status_code)   # 200
# print(resp.text)          # csv內容
resp_text = resp.text
file_name = request_url.split("/")[-1]
f = open(file_name, 'w')      # 將回應的文字寫出成 csv 檔案
f.write(resp_text)
f.close()

In [4]:
data = pd.read_csv('example/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [7]:
# 時間序列的資料處理
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [8]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [9]:
data = data.reindex(columns=columns)
data.head()

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


In [14]:
data.index = periods.to_timestamp('D', 'end')  # 時間戳  #?????????????????????
data.head()

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2


In [15]:
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [16]:
# pivot
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [17]:
# 重排兩欄資料
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,1.725301
1,1959-03-31 23:59:59.999999999,infl,0.000,1.863913
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.313647
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,0.123850
4,1959-06-30 23:59:59.999999999,infl,2.340,-1.372791
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.845055
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.685241
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.200075
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.783688
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-0.812428


In [18]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  1.863913  1.725301   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -1.372791  0.123850   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.200075  0.685241   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.838502 -0.812428   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.110058  2.980095   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.313647  
1959-06-30 23:59:59.999999999  0.845055  
1959-09-30 23:59:59.999999999  0.783688  
1959-12-31 23:59:59.999999999 -0.294059  
1960-03-31 23:59:59.999999999  0.409668

In [19]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


In [22]:
unstack = ldata.set_index(['date', 'item']).unstack('item')
unstack[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  1.863913  1.725301   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -1.372791  0.123850   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.200075  0.685241   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.838502 -0.812428   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.110058  2.980095   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -1.675827  2.017078   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -0.019386 -0.835793   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.313647  
1959-06-30 23:59:59.999999999  0.845055  
1959-09-30 23:59:59.999999999  0.783688  
1959-12-31 23:59:59.999999999 -0.294059  
1960-03-31 23:59:59.999999999  0.409668  
1960-06-30 23:59:59.999999999 -0.476221  
1960-09-30 23:59:59.999999999  0.367791

### "寬格式"旋轉成"長格式"

In [23]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6], 
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [25]:
# pandas.melt: 指定哪個欄是用來分組的
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [26]:
# pivot: 將資料重排成原來的樣子
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [27]:
# 把key資料搬回一般欄位中
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [28]:
# 指定一部分欄位當作轉換後的值欄位
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [29]:
# 不指定分組的欄位
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [30]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
